## Setup
This notebook was created and tested on an ml.m5.xlarge notebook instance.

Let's start by specifying:

- The S3 bucket and prefix that you want to use for training and model data. This should be within the same region as the Notebook Instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these. Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the boto regexp with a the appropriate full IAM role arn string(s).

In [2]:
# Define IAM role
import boto3
import re
from sagemaker import get_execution_role
role = get_execution_role()


import os

bucket = '' # set your bucket
key = '' # set your prefix
data_loc = 's3://{}/{}'.format(bucket, key)
train_file = data_loc+'/xxx.csv' # set your train file name
output_location = 's3://{}/{}'.format(bucket, 'model')


## Training the Linear-learner model
After setting training parameters, we kick off training, and poll for status until training is completed, which in this example, takes between 5 and 6 minutes.

In [18]:
from sagemaker.amazon.amazon_estimator import get_image_uri
container = get_image_uri(boto3.Session().region_name, 'linear-learner')

import sagemaker
from sagemaker.session import s3_input

sess = sagemaker.Session()

linear = sagemaker.estimator.Estimator(container,
                                       role, 
                                       train_instance_count=1, 
                                       train_instance_type='ml.m5.xlarge',
                                       output_path=output_location,
                                       sagemaker_session=sess)
linear.set_hyperparameters(feature_dim=11,
                           predictor_type='regressor',
                           mini_batch_size=200)

content_type = "text/csv"
train_data = s3_input(train_file, content_type=content_type)
linear.fit({'train': train_data})

## Create endpoint
After outputing the model, we creates the endpoint that serves up the model through specifying the name. The end result is an endpoint that can be validated and incorporated into production applications. This takes 9-11 minutes to complete.

In [9]:
linear_predictor = linear.deploy(initial_instance_count=1,
                                 instance_type='ml.m5.xlarge',
                                endpoint_name = 'linearendpoint')

-------------!

## Validate the model
Finally, we can now validate the model for use. They can obtain the endpoint, and generate predictions from the trained model using that endpoint.

In [14]:
from sagemaker.predictor import csv_serializer, json_deserializer
import pandas as pd

linear_predictor.content_type = 'text/csv'
linear_predictor.serializer = csv_serializer
linear_predictor.deserializer = json_deserializer

# using features to test
test = '1.0,142424.29207611087,40.86764526367188,39.803932189941406,40.750870408043156,221935.611618042,41.00096130371094,40.58504867553711,40.75204032648586,5446.0,3495.0'

result = linear_predictor.predict(test)
print(result)

{'predictions': [{'score': 783.171875}]}


## Delete Endpoint
Once you are done using the endpoint, you can use the following to delete it.

In [16]:
linear_predictor.delete_endpoint()